# Anomaly Detection Training Benchmark and Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/imx500_notebooks/pytorch/pytorch_efficient_anomaly_detection.ipynb)

### Overview

In this tutorial we demonstrate training, quantization and benchmarking of an anomaly detection model. The resulting model will be imx500 compatible

Classification models are powerful and reliable, but what if you have little or no examples of one of your classes, what if one of your classes contains too much unpredictable variation?

Here we go through the process of building, training and quantizing an anomaly detection model designed to solve exactly these problems.

Anomaly detection models are useful as they only require your typical images to train and can in theory determin anything that is not typical.

We use Efficient ad, one of the top performing anomaly detection models on the mvtec benchmark. benchmark leader board can be found [here](https://paperswithcode.com/sota/anomaly-detection-on-mvtec-ad)

This particular model uses the teacher student method. Where the student model is trained to both mimic the feature map output of a simple pre-trained CNN aswell as mimic the output of an auto encoder that is its self also trained on the normal images.

We use the [mvtec](https://www.mvtec.com/company/research/datasets/mvtec-ad) dataset to benchmark and train this model.

## Summary

In this tutorial we will cover for an anomaly detection model:

1. Mvtec Benchmark
2. Post training quantization.
3. Visulization
4. Training this model.

## Setup

### install relevant packages

In [ ]:
!pip install torch
!pip install torchvision
!pip install tifffile
!pip install tqdm
!pip install scikit-learn
!pip install Pillow
!pip install scipy
!pip install tabulate

Install MCT (if it’s not already installed). Additionally, in order to use all the necessary utility functions for this tutorial, we also copy [MCT tutorials folder](https://github.com/sony/model_optimization/tree/main/tutorials) and add it to the system path.

In [ ]:
import sys
import importlib

if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit
!git clone https://github.com/sony/model_optimization.git temp_mct && mv temp_mct/tutorials . && \rm -rf temp_mct
sys.path.insert(0,"tutorials")

Download and extract the mvtec benchmark dataset. This is used for both training and evaluation. Link below is a direct link from the mcvtec website. 

For more information on the Mvtec Benchmark dataset, visit: https://www.mvtec.com/company/research/datasets/mvtec-ad

In [ ]:
!mkdir mvtec_anomaly_detection
!wget https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz
!tar -xvf mvtec_anomaly_detection.tar.xz -C mvtec_anomaly_detection

Finally download the official mvtec benchmark. Link below is a direct link from the mcvtec website.

In [ ]:
!wget https://www.mydrive.ch/shares/60736/698155e0e6d0467c4ff6203b16a31dc9/download/439517473-1665667812/mvtec_ad_evaluation.tar.xz
!tar -xvf mvtec_ad_evaluation.tar.xz
!rm mvtec_ad_evaluation.tar.xz

## Model Quantization

### Download and Build Model

We have pretrained a model on the bottle dataset from mvtec. Here we will load the combined model from huggingface, a combination of (teacher, student, and autoenoder).

In [ ]:
from huggingface_hub import hf_hub_download
import tutorials.mct_model_garden.models_pytorch.Efficient_Anomaly_Det as efficient_ad

out_channels = 384

model_path = hf_hub_download(repo_id="SSI-DNN/Efficient_Anomaly_Detection", filename="efficientAD_bottle.pth")

teacher = efficient_ad.get_pdn_small(out_channels)
student = efficient_ad.get_pdn_small(2 * out_channels)
autoencoder = efficient_ad.get_autoencoder(out_channels)

model = efficient_ad.UnifiedAnomalyDetectionModel.load_model(model_path, teacher, student, autoencoder)

### Post training quantization using Model Compression Toolkit

To perform model quantization we require a representative dataset. Because of the specific requirements of anomaly detection models (the assumption that anomalous images are not seen) we restrict the representative dataset to as such.

In [ ]:
import model_compression_toolkit as mct
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from typing import Iterator, List
from tutorials.resources.utils.efficient_ad_utils import ImageFolderWithoutTarget, train_transform


def train_dataset_generator(train_loader: DataLoader) -> Iterator[List]:
    while True:
        for data, _ in train_loader:
            yield [data.numpy()]

def get_representative_dataset(n_iter: int, dataset_loader: Iterator[List]) -> Iterator[List]:
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield next(ds_iter)
    return representative_dataset

train_set = ImageFolderWithoutTarget(
    os.path.join('./mvtec_anomaly_detection', 'bottle', 'train'),
    transform=transforms.Lambda(train_transform))

train_loader = DataLoader(train_set, batch_size=4, shuffle=True)  # Ensure this matches your batch size and other DataLoader settings
train_dataset = train_dataset_generator(train_loader)
representative_dataset_gen = get_representative_dataset(n_iter=20, dataset_loader=train_dataset)

# Set target platform capabilities
tpc = mct.get_target_platform_capabilities(fw_name="pytorch", target_platform_name='imx500', target_platform_version='v1')

# Perform post training quantization
quant_model, _ = mct.ptq.pytorch_post_training_quantization(in_module=model,
                                                            representative_data_gen=representative_dataset_gen,
                                                            target_platform_capabilities=tpc)

## Model Export

This model can be converted to run on imx500.


In [ ]:
import model_compression_toolkit as mct

mct.exporter.pytorch_export_model(model=quant_model,
                                  save_model_path='./quant_model.onnx',
                                  repr_dataset=representative_dataset_gen)

## Float Benchmark

Mvtec benchmark provides its own code. we first need to run the model on the test images and save the output heat maps and anomaly confidence. then run mvtecs benchmark.

In [ ]:
#We first need to calculate the normalisation values

q_st_start, q_st_end, q_ae_start, q_ae_end = map_normalization(
    validation_loader=train_loader, teacher=teacher, student=student,
    autoencoder=autoencoder, teacher_mean=teacher_mean,
    teacher_std=teacher_std, desc='Final map normalization')

In [ ]:
from tutorials.mct_model_garden.anomaly_eval import benchmark
benchmark(model, 'mvtec_ad', q_st_start, q_st_end, q_ae_start, q_ae_end)

### Mvtec benchmark

This results in a classification accuracy AU-ROC and a segmentation accuracy AU-PRO

In [ ]:
!python ./mvtec_ad_evaluation/evaluate_experiment.py --dataset_base_dir './mvtec_anomaly_detection/' --anomaly_maps_dir './output/anomaly_maps/mvtec_ad/' --output_dir './output/metrics/mvtec_ad/' --evaluated_objects bottle

## Quantized model benchmark


In [ ]:
from tutorials.mct_model_garden.anomaly_eval import benchmark
benchmark(quant_model, 'mvtec_ad_quant')

In [ ]:
!python ./mvtec_ad_evaluation/evaluate_experiment.py --dataset_base_dir './mvtec_anomaly_detection/' --anomaly_maps_dir './output/anomaly_maps/mvtec_ad_quant/' --output_dir './output/metrics/mvtec_ad/' --evaluated_objects bottle

## Anomaly Map Visulization

We can visulize the heatmap of the predicted anomalies with the code below. Here red spots indicate locations with a high likely hood of defect, based on its training images. We also print the models prediction.


In [ ]:
from tutorials.resources.utils.efficient_ad_utils import visualize_anomalies
import os
name = 'visulize'
dataset_path = './mvtec_anomaly_detection'
test_output_dir = os.path.join('output', 'anomaly_maps',
                                name, 'bottle', 'test')
model.eval()
visualize_anomalies(model, dataset_path, test_output_dir)

## Conclusion

In this notebook we provide examples on how to quantize and benchmark the latest anomaly detection model as well as providing code to visulize the models output.

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.